# Original Losses Value Bar Charts

In [3]:

import pandas as pd
import matplotlib.pyplot as plt
import os

# ================= setting area ==================
dimensions = ['1D', '2D', '3D', '4D']
excluded_conditions = ['Neumann']   # All Neumann loss values are '0'
# =================================================

full_conditions = ['Unsupervised', 'Initial', 'Dirichlet', 'Neumann', 'FB_Init', 'FB_Dir', 'FB_Neu', 'Free Boundary', 'Total']
filtered_conditions = [attr for attr in full_conditions if attr not in excluded_conditions]

colors = ['#1f77b4', '#ff7f0e']
bar_width = 0.4
plt.rcParams.update({
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10
})

for dim in dimensions:

    fig = plt.figure(figsize=(10, 6))
    file_path = os.path.join('mean&median_losses', f'{dim}.xlsx')
    df = pd.read_excel(file_path, header=1)

    mean_values = []
    median_values = []
    for attr in filtered_conditions:
        mean_values.append(df[df['Seed'] == 'Mean'][attr].values[0])
        median_values.append(df[df['Seed'] == 'Median'][attr].values[0])

    ax = fig.add_subplot(111)
    x = range(len(filtered_conditions))

    bars1 = ax.bar(x, mean_values, width=bar_width, color=colors[0], label='Mean')
    bars2 = ax.bar([i + bar_width for i in x], median_values, width=bar_width, color=colors[1], label='Median')

    ax.set_title(f'{dim} Model Loss Comparison', pad=15)
    ax.set_xticks([i + bar_width / 2 for i in x])
    ax.set_xticklabels(filtered_conditions, rotation=30, ha='right')
    ax.set_ylabel('Loss Value')
    ax.grid(True, axis='y', alpha=0.3)
    ax.legend()

    # for bar in bars1 + bars2:
    #     height = bar.get_height()
    #     ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2e}', ha='center', va='bottom', fontsize=8)

    output_path = os.path.join('mean&median_losses', f'{dim}_loss_comparison.png')
    plt.savefig(output_path, bbox_inches='tight', dpi=150)
    plt.close()


# Details

In [46]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import os

# ================= setting area ==================
dimensions = ['1D', '2D', '3D', '4D']
excluded_conditions = ['Neumann']   # All Neumann loss values are '0'
threshold_config = {
    '1D': 0.0003,
    '2D': 0.0015,
    '3D': 0.0025,
    '4D': 0.25
}
lower_ratio = 0.7
# =================================================

full_conditions = ['Unsupervised', 'Initial', 'Dirichlet', 'Neumann', 'FB_Init', 'FB_Dir', 'FB_Neu', 'Free Boundary', 'Total']
filtered_conditions = [attr for attr in full_conditions if attr not in excluded_conditions]

# 带阈值参数的坐标转换函数
def y_transform(y, threshold, lower_ratio):
    if y <= threshold:
        return y * (lower_ratio / threshold)
    else:
        return lower_ratio + (y - threshold) * ((1 - lower_ratio) / (max_y - threshold))

def y_inverse(transformed_y, threshold, lower_ratio):
    if transformed_y <= lower_ratio:
        return transformed_y * threshold / lower_ratio
    else:
        return threshold + (transformed_y - lower_ratio) * (max_y - threshold) / (1 - lower_ratio)

colors = ['#1f77b4', '#ff7f0e']
bar_width = 0.4
plt.rcParams.update({
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10
})

for dim in dimensions:

    threshold = threshold_config[dim]
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111)
    file_path = os.path.join('mean&median_losses', f'{dim}.xlsx')
    df = pd.read_excel(file_path, header=1)

    mean_values = []
    median_values = []
    for attr in filtered_conditions:
        mean_values.append(df[df['Seed'] == 'Mean'][attr].values[0])
        median_values.append(df[df['Seed'] == 'Median'][attr].values[0])

    max_y = max(mean_values + median_values)
    transformed_mean = [y_transform(y, threshold, lower_ratio) for y in mean_values]
    transformed_median = [y_transform(y, threshold, lower_ratio) for y in median_values]

    x = np.arange(len(filtered_conditions))
    bars1 = ax.bar(x - bar_width/2, transformed_mean, width=bar_width, color=colors[0], label='Mean')
    bars2 = ax.bar(x + bar_width/2, transformed_median, width=bar_width, color=colors[1], label='Median')

    lower_ticks = np.linspace(0, threshold, 6)
    upper_ticks = np.linspace(threshold, max(threshold * 1.1, max_y), 4)

    all_ticks = list(lower_ticks) + list(upper_ticks[1:])
    transformed_ticks = [y_transform(t, threshold, lower_ratio) for t in all_ticks]

    ax.set_yticks(transformed_ticks)
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{y_inverse(x, threshold, lower_ratio):.4f}"))

    ax.axhline(y = y_transform(threshold, threshold, lower_ratio), color='gray', linestyle='--', linewidth=0.8)
    ax.text(0.05, y_transform(threshold, threshold, lower_ratio) * 1.07,
           f'Threshold: {threshold}', ha='left', va='top', color='gray', backgroundcolor='white')

    ax.set_title(f'{dim} Model Loss Comparison\n(Threshold={threshold})', pad=15)
    ax.set_xticks(x)
    ax.set_xticklabels(filtered_conditions, rotation=30, ha='right')
    ax.set_ylabel('Loss Value')
    ax.grid(True, axis='y', alpha=0.3)
    ax.legend()

    output_path = os.path.join('mean&median_losses', f'{dim}_threshold.png')
    plt.savefig(output_path, bbox_inches='tight', dpi=150)
    plt.close()
